In [ ]:
# from transformers import TrainingArguments

# # 학습 인자 정의
# training_args = TrainingArguments(
#     output_dir=path,
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     logging_dir="logs",
# )

# # 학습 인자 저장
# training_args.save(path+"training_args.bin")


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.6 MB/s eta 0:00:00


In [ ]:
#데이터셋 로드
import pandas as pd
chatbot_data = pd.read_excel('/content/8개의 감정데이터셋_v9.xlsx')

In [ ]:
# 데이터셋 라벨링

chatbot_data.loc[(chatbot_data['Emotion'] == "우울한"), 'Emotion'] = 0
chatbot_data.loc[(chatbot_data['Emotion'] == "기쁜"), 'Emotion'] = 1
chatbot_data.loc[(chatbot_data['Emotion'] == "화가 나는"), 'Emotion'] = 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픈"), 'Emotion'] = 3
chatbot_data.loc[(chatbot_data['Emotion'] == "편안한"), 'Emotion'] = 4
chatbot_data.loc[(chatbot_data['Emotion'] == "걱정스러운"), 'Emotion'] = 5
chatbot_data.loc[(chatbot_data['Emotion'] == "신이 난"), 'Emotion'] = 6
chatbot_data.loc[(chatbot_data['Emotion'] == "충만한"), 'Emotion'] = 7



In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(chatbot_data, test_size=0.2, random_state=0)

In [ ]:
train_texts = []
for q in train_df['Sentence']  :
    #data = []
    train_texts.append(q)
    #data.append(str(label))

    #data_list.append(data)

In [ ]:
train_labels = []
for q in train_df['Emotion']  :
    #data = []
    train_labels.append(q)
    #data.append(str(label))

    #data_list.append(data)

In [ ]:
test_texts = []
for q in test_df['Sentence']  :
    #data = []
    test_texts.append(q)
    #data.append(str(label))

    #data_list.append(data)

In [ ]:
test_labels = []
for q in test_df['Emotion']  :
    #data = []
    test_labels.append(q)
    #data.append(str(label))

    #data_list.append(data)

In [ ]:
len(train_labels)

91096

In [ ]:
len(test_labels)

22775

In [ ]:
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [ ]:
#max_len = 64
batch_size = 64
num_epochs = 50
learning_rate =  2e-5

In [ ]:
## Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 7
#max_grad_norm = 1
#log_interval = 200
learning_rate =  5e-5

In [ ]:
#batch_size = 128  # 배치 사이즈는 직접 지정해야 합니다.
#batch_size=64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=8) # 0, 1로 분류하기 때문에 레이블은 2개로 지정합니다.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#print(torch.cuda.memory_summary())


In [ ]:
def calc_accuracy(outputs, labels):
    # 모델의 출력 중에서 예측된 클래스를 추출합니다.
    predicted_classes = torch.argmax(outputs.logits, dim=1)

    # 정확하게 분류된 클래스 수를 계산합니다.
    correct_predictions = (predicted_classes == labels).sum().item()

    # 전체 데이터 포인트 수로 나누어 정확도를 계산합니다.
    accuracy = correct_predictions / labels.size(0)

    return accuracy


In [ ]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup, AdamW
import torch
from tqdm.auto import tqdm

# 모델 로드
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=8)

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 학습률 스케줄러 설정
t_total = len(train_loader) * num_epochs
warmup_steps = int(t_total * warmup_ratio)  # warmup_ratio = 0.1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

# 손실 함수 설정
criterion = torch.nn.CrossEntropyLoss()

# 정확도 계산 함수
def calc_accuracy(logits, labels):
    max_vals, max_indices = torch.max(logits, 1)
    acc = (max_indices == labels).sum().item() / max_indices.size(0)
    return acc

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_acc = 0.0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        train_acc += calc_accuracy(logits, labels)

    average_loss = total_loss / len(train_loader)
    average_acc = train_acc / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")
    print(f"Epoch {epoch+1} Train Accuracy: {average_acc:.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/712 [00:00<?, ?it/s]

In [ ]:
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

# 모델 로드
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=8)

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=learning_rate)

# 학습률 스케줄러 설정
t_total = len(train_loader) * num_epochs
warmup_steps = int(t_total * warmup_ratio)  # 예: warmup_ratio = 0.1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

# 손실 함수 설정
criterion = torch.nn.CrossEntropyLoss()

# 정확도 계산 함수
def calc_accuracy(X, Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy() / max_indices.size()[0]
    return train_acc

# 학습 루프
for epoch in range(num_epochs):
    model.train()  # 훈련 모드
    total_loss = 0
    train_acc = 0.0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()  # 학습률 스케줄러 스텝

        train_acc += calc_accuracy(outputs.logits, labels)

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")
    print(f"Epoch {epoch+1} train acc {train_acc / len(train_loader):.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/712 [00:00<?, ?it/s]

In [ ]:
from tqdm.auto import tqdm # 반복문이 얼마나 진행되었는지 알 수 있도록 프로그레스바를 표시합니다.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용이 가능한 경우 설정


#num_epochs = 3
#learning_rate = 2e-5 #2e-5는 0.00002
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()

model.to(device) # GPU 사용이 가능한 경우

for epoch in range(num_epochs):
    model.train() # 훈련 모드 지정
    total_loss = 0
    train_acc = 0.0
    #test_acc = 0.0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        #input_ids = batch['input_ids']
        attention_mask = batch['attention_mask'].to(device)
        #attention_mask = batch['attention_mask']
        #labels=batch['labels']
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_acc += calc_accuracy(outputs, labels)
    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")
    print("epoch {} train acc {}".format(epoch+1, train_acc / len(train_loader)))

  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 1/50 - Average Loss: 1.8007
epoch 1 train acc 0.28022223197565543


  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 2/50 - Average Loss: 1.6699
epoch 2 train acc 0.3446607268258427


  0%|          | 0/1424 [00:00<?, ?it/s]

...

  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 36/50 - Average Loss: 0.7309
epoch 36 train acc 0.7044929190074906


  0%|          | 0/1424 [00:00<?, ?it/s]

Epoch 37/50 - Average Loss: 0.7271
epoch 37 train acc 0.7055572624063671


  0%|          | 0/1424 [00:00<?, ?it/s]

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        correct_predictions += torch.sum(predicted_labels == labels).item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
test_loader

In [ ]:
predicted_labels

In [ ]:
print(correct_predictions)

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 현재경로 /content/

import os
os.chdir('/content/drive/MyDrive/V9_epoch50/')
os.getcwd()

path = '/content/drive/MyDrive/V9_epoch50/'

model.save_pretrained(path, save_config=True, save_model=True)

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/V9_epoch50/pytorch_model.bin")

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
input_text = '별로'
input_encoding = tokenizer.encode_plus(
    input_text,
    truncation=True,
    padding=True,
    return_tensors='pt'
)

input_ids = input_encoding['input_ids'].to(device)
attention_mask = input_encoding['attention_mask'].to(device)

model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    _, predicted_labels = torch.max(outputs.logits, dim=1)
predicted_labels = predicted_labels.item()

print(predicted_labels)

0
